# Extract Tree Cover from Hansen 2013

Run this notebook to merge all the separate files downloaded from GEE.


## Google Earth Enginge Download Code


```javascript
var dataset = ee.Image("UMD/hansen/global_forest_change_2022_v1_10");
var treeCoverVisParam = {
  bands: ["treecover2000"],
  min: 0,
  max: 100,
  palette: ["black", "green"],
};
Map.addLayer(dataset, treeCoverVisParam, "tree cover");

var treeLossVisParam = {
  bands: ["lossyear"],
  min: 0,
  max: 22,
  palette: ["yellow", "red"],
};
Map.addLayer(dataset, treeLossVisParam, "tree loss year");

// User input
var exportDescription = "hansen2013_treecover2000_france"; // Description of the export task
var exportFolder = "gee_exports"; // Google Drive folder to export the image
var exportFileNamePrefix = "hansen2013_treecover2000_france"; // File name prefix for the exported image
var exportScale = 30; // Scale in meters; change if needed
var exportMaxPixels = 1e9; // Adjust according to the size of your region

// Define the bounding box for metropolitan France
var franceBounds = ee.Geometry.Rectangle([-5.14, 41.33, 9.56, 51.09]);

// Function to split the bounding box into a specified number of rectangles
function splitIntoRectangles(bounds, numRectangles) {
  var boundsCoords = ee.List(bounds.coordinates().get(0));
  var xMin = ee.Number(ee.List(boundsCoords.get(0)).get(0));
  var xMax = ee.Number(ee.List(boundsCoords.get(2)).get(0));
  var yMin = ee.Number(ee.List(boundsCoords.get(0)).get(1));
  var yMax = ee.Number(ee.List(boundsCoords.get(2)).get(1));

  var width = xMax.subtract(xMin);
  var height = yMax.subtract(yMin);

  var rectWidth = width.divide(Math.sqrt(numRectangles));
  var rectHeight = height.divide(Math.sqrt(numRectangles));

  var rectangles = [];

  for (var i = 0; i < Math.sqrt(numRectangles); i++) {
    for (var j = 0; j < Math.sqrt(numRectangles); j++) {
      var rect = ee.Geometry.Rectangle([
        xMin.add(rectWidth.multiply(i)),
        yMin.add(rectHeight.multiply(j)),
        xMin.add(rectWidth.multiply(i + 1)),
        yMin.add(rectHeight.multiply(j + 1)),
      ]);
      rectangles.push(rect);
    }
  }

  return rectangles;
}

// Create 10 rectangles that cover the bounding box
var rectangles = splitIntoRectangles(franceBounds, 10);

// Display the rectangles on the map and prepare export tasks
rectangles.forEach(function (rect, index) {
  var layerName = "Rectangle " + (index + 1);
  Map.addLayer(rect, { color: "blue" }, layerName);

  // Set up the export task
  Export.image.toDrive({
    image: dataset.select(["treecover2000"]),
    description: exportDescription + "_part" + (index + 1),
    folder: exportFolder,
    fileNamePrefix: exportFileNamePrefix + "_part" + (index + 1),
    scale: exportScale,
    region: rect,
    maxPixels: exportMaxPixels,
  });

  // Note: The export task is automatically queued upon calling Export.image.toDrive
});

// Center the map over France
Map.centerObject(franceBounds, 6);
```


## Imports


In [ ]:
# Magic
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Libraries
import geopandas as gpd
import rasterio
from rasterio.merge import merge
from rasterio.crs import CRS
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
import sys
from pyprojroot import here

sys.path.insert(0, "../../src")
from run_mp import *
from utilities import *

import chime
chime.theme("mario")

## Merge Raster Parts


In [ ]:
def merge_raster_parts(myvar):
    display(f"Extraction for file {myvar}")
    # ! List all tif files of desired variable -----------------------------------------
    all_files = glob.glob(f"../../data/hansen2013/{myvar}/*.tif")
    print("- Number of files: ", len(all_files))
    print(all_files)

    # ! Merge all files into one --------------------------------------------------------
    folder = here(f"data/hansen2013/{myvar}")
    os.makedirs(folder, exist_ok=True)
    filename = here(f"data/hansen2013/{myvar}_merged_in_python.tif")

    # List for the data
    src_files_to_mosaic = []
    print("- Loading all files into a list...")
    for file in tqdm(all_files):
        src = rasterio.open(file)
        src_files_to_mosaic.append(src)

    # Merge function returns a single mosaic array and the transformation info
    print("- Merge parts into one...")
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Copy the metadata
    out_meta = src.meta.copy()

    # Update the metadata
    out_meta.update(
        {
            "driver": "GTiff",
            "height": mosaic.shape[1],
            "width": mosaic.shape[2],
            "transform": out_trans,
            "crs": src.crs.to_epsg(),
        }
    )

    # Save the merged raster as a TIFF file
    with rasterio.open(
        filename,
        "w",
        **out_meta,
    ) as dest:
        dest.write(mosaic)

In [ ]:
merge_raster_parts("treecover2000")
# merge_raster_parts("lossyear")

In [ ]:
# Quality Control.
# ! Outcommented because it can take up to 1 min!
# Open the saved TIFF file and plot it
# with rasterio.open(filename) as src:
#     # Read the raster data
#     raster_data = src.read(1)

#     # Display the raster data
#     plt.imshow(raster_data, cmap="gray")
#     plt.colorbar()
#     plt.show()

## Extract Data at Sites

- Extraction done in QGIS, was easier...


In [ ]:
lossyear = pd.read_csv(here("qgis/hansen2013/hansen2013_lossyear.csv"))
treecover = pd.read_csv(here("qgis/hansen2013/hansen2013_treecover.csv"))

df = pd.merge(
    lossyear,
    treecover,
    how="outer",
)
df = df.rename(columns={"lossyear1": "lossyear", "treecover1": "treecover"})[
    ["idp", "lossyear", "treecover"]
]
df

## Save Final Data


In [ ]:
df.to_feather(here("data/final/predictor_datasets/treecover.feather"))
df
chime.success()

# Forest Cover Map


In [ ]:
import sys

sys.path.insert(0, "../../src")
from imports import *

init_notebook()

import rasterio
import numpy as np
import pandas as pd
from rasterio.transform import Affine

import matplotlib.dates as mdates
from sklearn.linear_model import LinearRegression
from matplotlib.lines import Line2D
import matplotlib.lines as mlines
from scipy.stats import mode

In [ ]:
df = aggregate_raster_to_csv(
    "../../data/hansen2013/treecover2000_merged_in_python-clipped_mask_france.tif",
    "../../data/hansen2013/treecover2000_merged_in_python-clipped_mask_france.csv",
    2 * 1000,
    save_file=False,
    verbose=True,
)

In [ ]:
make_map_for_temp_prec_cover(
    df,
    "treecover",
    pixel_res=500j,
    textsize=20,
    contour_levels=11,
    filepath="./forest_cover/forest_cover_map.png",
    tick_interval=1,
)